In [ ]:
# conda install -c conda-forge findspark

### De aqui en adelante esta el codigo para las funciones solicitadas

## Funcion de agrupacion (group-by)
Esta funcion recibe como parametros un dataframe y un diccionario anidado con el nombre de la columna resultado de la agrupacion (como clave) y como valor otro diccionario que tiene como primer elemento la clave tipo y como valor el tipo de agrupacion y como segundo argumento la clave columna y como valor el nombre de la columna que se quiere agrupar , esta funcion retorna un nuevo dataframe con el resultado de la agrupacion.

In [2]:
# Definir la función para agrupar el DataFrame por cliente
def group_df(dataframe, diccionario_agrupacion, group_by_columns):# se recibe un tercer parametro(lista) para saber por cuales columnas agrupar
    expresiones_agrupacion = []
    for nombre_columna, info_agrupacion in diccionario_agrupacion.items():
        
        tipo_agrupacion = info_agrupacion['type']
        columna_agrupacion = info_agrupacion['column']
        if tipo_agrupacion == 'count':
            expresiones_agrupacion.append(F.count(columna_agrupacion).alias(nombre_columna))
        elif tipo_agrupacion == 'sum':
            expresiones_agrupacion.append(F.sum(columna_agrupacion).alias(nombre_columna))
            
    dataframe_agrupado = dataframe.groupBy(*group_by_columns).agg(*expresiones_agrupacion)
    return dataframe_agrupado



In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# SparkSession
spark = SparkSession.builder \
    .appName("Ejemplo PySpark") \
    .getOrCreate()

# DataFrame de ejemplo 
datos = [("1", "Cliente1", 100, 5),
         ("2", "Cliente2", 200, 3),
         ("1", "Cliente1", 300, 7),
         ("2", "Cliente2", 400, 4),
         ("2", "Cliente2", 500, 2)]
df = spark.createDataFrame(datos, ["idcliente", "nombre_cliente", "monto", "cantidadProductos"])

# para ver el DataFrame original
print("DataFrame original:")
df.show()

# diccionario de agrupación ejemplo
diccionario_agrupacion = {
    "suma_monto": {
        "type": "sum",
        "column": "monto"
    },
    "suma_cantidad_productos": {
        "type": "sum",
        "column": "cantidadProductos"
    }
}
# Probar la función agrupando el DataFrame por la columna segun la lista pasada como parametro
# con el diccionario de agrupación
df_agrupado = group_df(df, diccionario_agrupacion, ["nombre_cliente"])

# Mostrar el DataFrame agrupado
print("DataFrame agrupado por cliente:")
df_agrupado.show()

24/02/22 22:55:51 WARN Utils: Your hostname, merlin-ThinkPad-T470-W10DG resolves to a loopback address: 127.0.1.1; using 192.168.0.9 instead (on interface wlp4s0)
24/02/22 22:55:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/22 22:55:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


DataFrame original:


+---------+--------------+-----+-----------------+
|idcliente|nombre_cliente|monto|cantidadProductos|
+---------+--------------+-----+-----------------+
|        1|      Cliente1|  100|                5|
|        2|      Cliente2|  200|                3|
|        1|      Cliente1|  300|                7|
|        2|      Cliente2|  400|                4|
|        2|      Cliente2|  500|                2|
+---------+--------------+-----+-----------------+

DataFrame agrupado por cliente:


+--------------+----------+-----------------------+
|nombre_cliente|suma_monto|suma_cantidad_productos|
+--------------+----------+-----------------------+
|      Cliente1|       400|                     12|
|      Cliente2|      1100|                      9|
+--------------+----------+-----------------------+



24/02/22 22:56:06 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
